This notebook aims at giving evaluation on my model. 
1. Choose 5-6 pictures of generated images to show how training progresses (for example - epoch 0, 100, 200, 300, 400, 500).
2. A batch of real images, a batch of the gradients from an alternate class for these images, and the modified images the discriminator incorrectly classifies.
3. Synthetic images maximizing the class output. One for the discriminator trained without the generator and one for the discriminator trained with the generator.
4. Synthetic images maximizing a particular layer of features. Do this for at least two different layers (for example - layer 4 and layer 8.)
5. Report your test accuracy for the two discriminators.

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Plot images
def custom_plot(samples):
    fig = plt.figure(figsize=(10, 10))
    gs = gridspec.GridSpec(10, 10)
    gs.update(wspace=0.02, hspace=0.02)
    
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis("off")
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        plt.imshow((sample*255).astype(np.uint8))
    return fig

In [ ]:
# 2
transform_test = transforms.Compose([
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

testset = torchvision.datasets.CIFAR10(root='./', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)
testloader = enumerate(testloader)

model = torch.load('best_GAN_Baseline_Discriminator.ckpt').to(device)
model.eval()

batch_idx, (X_batch, Y_batch) = testloader.next()
X_batch = X_batch.to(device)
Y_batch_alternate = (Y_batch + 1)%10
Y_batch_alternate = Y_batch_alternate.to(device)
Y_batch = Y_batch.to(device)

samples = X_batch.cpu().numpy()
samples += 1
samples /= 2
samples = samples.transpose(0,2,3,1)
fig = custom_plot(samples)
plt.savefig("HW6/Visualization/real_images.png", bbox_inches="tight")
plt.close(fig)

_, output = model(X_batch)
prediction = output.data.max(1)[1] # first column has actual prob.
accuracy = ( float( prediction.eq(Y_batch.data).sum() ) /float(batch_size))*100.0
print(accuracy)

In [ ]:
# 3
